## Importing Data
### 1.基本介绍
Tensorflow提供的tf.data接口，允许你通过简单可重复使用的部件构建一个复杂的输入流水线。
<br>(翻译成流水线感觉有点怪，我也不知道Input Pipeline该怎么翻译好)
<br>例如，一个图片模型的输入，也许会从分布式系统中合并文件后读取，在每个图片上加入随机干扰，然后随机选择图片加入到一个batch中进行训练。下一个模型的流水线或许设计从原始数据中提取符号，将它们通过查找表转化为<a>Embedding Identifiers</a>,将不同长度的序列合为一个batch。
<br>tf.data使处理大量数据，不同的数据形式，以及复杂的转化方式变得容易。

#### tf.data 为开发人员提供两个抽象接口：
<BR> 一个tf.data.Dataset代表一系列的元素，其中每个元素包含一个或者多个<a>Tensor</a>对象.
<br> 例如，一个图像流水线，一个元素或许是一个训练样例，其中包含一对Tensor，分别代表图片数据以及一个标签，有两种不同的方式来创建一个数据集：
* 创建一个数据源(e.g.Dataset.from_tensor_slices())，通过tf.Tensor对象来构建数据集。
* 应用转换(e.g.Dataset.batch())从tf.data.Dataset对象中构件数据集

此外，tf.data.Iterator提供从数据集提取元素的主要方式，当该方法被执行时，通过Iterator.get_next()返回数据集的下一个元素。在输入流水线意见你的模型之间，最简单的迭代器是“one-shor iterator”,它与特定的数据集绑定后，迭代运行一次。对于更多复杂的使用方法，Iterator.initializer操作使你可以在不同的数据集上重新初始化迭代器，例如，在同一个程序上，迭代训练集和验证集多次。

### 2.基本机制
接下来这部分介绍关于创建不同类型Dataset以及Iterator的基本部件，以及如何从它们提取数据。
<BR>
开始一个输入流水线，首先要定义一个数据源，例如，从内存中的Tensor构建一个数据集，一个可以使用tf.data.Dataset.from_tensors()或者tf.data.Dataset.from_tensor_slices()。再或者，如果你的输入数据以TFRecord格式存储在磁盘中，你可以创建tf.data.TFRecordDataset.
<BR>
一旦你有一个Dataset对象，你可以将它转化为新的数据集通过tf.data.Dataset对象提供的一系列方法。例如，你可以在每个元素上应用Dataset.map(),也可以一次性转换多个元素(Dataset.batch()).更多方法参考<a href = "https://www.tensorflow.org/api_docs/python/tf/data/Dataset?hl=zh-cn">tf.data.Dataset</a>
<BR>
最常见的使用数据集的方法是，实现一个Iterator对象，每次提供数据集的一个元素；tf.data.Iterator提供两种操作：
* Iterator.initializer:初始化Iterator状态
* Iterator.get_next():返回tf.Tensors对象，包含数据集的下一个元素

#### 2.1 数据集结构
每个数据集包含的元素具有类似的结构。一个元素具有一个或者多个tf.Tensor对象，称为组件。每个组件具有一个tf.Dtype表示Tensor包含的元素的数据类型，tf.TensorShape表示每个元素的静态大小。
<br>Dataset.output_type以及Dataset.output_shapes方法允许我们查看数据集的数据类型以及大小，示例代码如下：

In [3]:
import tensorflow as tf
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

<dtype: 'float32'>
(10,)


此外，每个元素可以包含一个或者多个包装在元组中的tensors，甚至是压缩的多个Tensors。

In [4]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"

(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))
(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


同时，如果有需要的话，可以给element的每个部件命名

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(
   {"a": tf.random_uniform([4]),
    "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes)  # ==> "{'a': (), 'b': (100,)}"

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


以及，将当前数据集通过调用Dataset.map(),Dataset.flat_map()以及Dataset.filter()等方法，转换为自己需要的数据集（注意，该转换应用在每个元素）
示例伪代码如下👇

dataset1 = dataset1.map(lambda x: ...)

dataset2 = dataset2.flat_map(lambda x, y: ...)

dataset3 = dataset3.filter(lambda x, (y, z): ...)

#### 2.2 创建迭代器
一旦你已经创建一个数据集来表示你的输入数据，那么下一步就是创建一个迭代器来获取数据集的元素。tf.data的API接口目前支持一下迭代器，复杂程度依次递增：
* one-shot
* initializable
* reinitializable
* feedable
(这篇tutorial仅介绍最常用的one-shot Iterator，其他请查看<a href="https://www.tensorflow.org/programmers_guide/datasets?hl=zh-cn">官方文档</a>)

#### One-shot
one-shot迭代器是最简单的一种，它仅支持迭代一次数据，没有特定的初始化，one-shot迭代器几乎可以处理所有基于队列的输入形式，但是它们不支持参数。
<br>示例如下：

In [10]:
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()
for i in range(10):
    value = sess.run(next_element)
    print(value)
    assert i == value

0
1
2
3
4
5
6
7
8
9


### ✨提示：目前来说，one-shot是唯一一个在Estimator中可使用的迭代器

#### 2.3 使用迭代器的返回值
Iterator.get_next()方法返回一个或者多个tf.Tensors对象，对应相应的下一个元素。

In [12]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element)

sess.run(iterator.initializer)
print(sess.run(result))  # ==> "0"
print(sess.run(result))  # ==> "2"
print(sess.run(result))  # ==> "4"
print(sess.run(result))  # ==> "6"
print(sess.run(result))  # ==> "8"


0
2
4
6
8
End of dataset


如果迭代器到达数据底部，再执行Iterator.get_next()会引发tf.errors.OutOfRangeError错误。在这一点后，迭代器将出于不可用的状态，如果你想在未来继续使用必须再次初始化。

In [13]:
try:
    sess.run(result)
except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"

End of dataset


换一种更为常见的写法：

In [15]:
sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(result))
    except tf.errors.OutOfRangeError:
        break

0
2
4
6
8


如果数据集的每个元素都有嵌套结构，那么Iterator.get_next()的返回值将是多个同一嵌套结构的tf.Tensors：

In [22]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random_uniform([4]), tf.random_uniform([4, 100])))
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

iterator = dataset3.make_initializable_iterator()

sess.run(iterator.initializer)
test = iterator.get_next()
print(test)

(<tf.Tensor 'IteratorGetNext_10:0' shape=(10,) dtype=float32>, (<tf.Tensor 'IteratorGetNext_10:1' shape=() dtype=float32>, <tf.Tensor 'IteratorGetNext_10:2' shape=(100,) dtype=float32>))


### 3.读入输入数据
#### 3.1 使用Numpy数组
如果你的所有输入数据都已存入在内存，最简单的创建数据集的方法就是，用Dataset.from_tensor_slices()将它们转化为tf.Tensors对象

In [ ]:
import numpy as np
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
    features = data["features"]
    labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

注意以上代码片段将features和labels数列作为tf.constant()嵌入在你的计算图(<a href="">graph</a>)中.在一个小的数据集中可以工作良好，但是浪费内存----因为数列中的内容需要被复制几次，且tf.GraphDef协议限制缓冲最大空间为2GB。
<BR>
作为替换,你可以定义Dataset为tf.placeholder(),在初始化数据集的Iterator时，再喂入Numpy arrays

In [ ]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
    features = data["features"]
    labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on `dataset`...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})

#### 3.2 使用TFRecord数据
tf.data提供对各种文件格式的支持，所以你可以处理在内存中无法存下的大规模数据集。例如，TFRecord文件格式是一种简单的二进制格式，tf.data.TFRecordDataset类允许你使用一个或多个TFRecord文件作为输入流水线的一部分。
<br>示例如下：

In [ ]:
# Creates a dataset that reads all of the examples from two files.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)

#### 3.3使用文本数据
许多数据集都分布在一个或者多个text文件中。tf.data.TextLIneDataset提供一种简易的方式从一个或者多个text文件中读入行数据。假如有一个或者多个文件，一个TextLineDataset将会为每一个文件产生一行String元素。TextLineDataset接收文件名字作为tf.Tensor,所以你可以通过将其传入tf.placeholder(tf.string)来参数化。
<br>示例如下 👇

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]
dataset = tf.data.TextLineDataset(filenames)

默认来说，一个TextLineDataset以行读入文件，但或许太过理想化，例如文件开始于标题行，或者包含一些评论，你可以通过调用Dataset.skip()以及Dataset.filter()来移除这些行，将这些转化分别应用在读入的文件中，我们可以调用Dataset.flat_map()方法，来为每个文件创建一个压缩版数据集。

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]

dataset = tf.data.Dataset.from_tensor_slices(filenames)

# Use `Dataset.flat_map()` to transform each file as a separate nested dataset,
# and then concatenate their contents sequentially into a single "flat" dataset.
# * Skip the first line (header row).
# * Filter out lines beginning with "#" (comments).
dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))))

### 4.用Dataset.map()预处理数据
Dataset.map(f)转化函数通过在输入数据集的每个元素上应用给定的函数f来产生新的数据集，这基于<a href = "https://en.wikipedia.org/wiki/Map_(higher-order_function)">map</a>，是一种常见的函数编程语言，函数f获取一个输入单元元素的tf.Tensors对象，并返回新数据集的单独元素。它的实现使用了标准的Tensorflow操作，从一个元素转化为另一个。
<BR>这一部分内容主要介绍如何使用Dataset.map()

#### 4.1 解析tf.Example协议缓存信息
许多输入流水线从TFRecord格式的文件中提取tf.train.Example协议缓存信息。每一个tf.train.Example记录包含一个或者多个“特征”，之后输入流水线通常会将这些特征转换为Tensors.
<br>示例代码如下👇

In [ ]:
# Transforms a scalar string `example_proto` into a pair of a scalar string and
# a scalar integer, representing an image and its label, respectively.
def _parse_function(example_proto):
    features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
              "label": tf.FixedLenFeature((), tf.int32, default_value=0)}
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features["image"], parsed_features["label"]

# Creates a dataset that reads all of the examples from two files, and extracts
# the image and label features.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)

#### 4.2解码图片数据集并重写调整大小
当用真实世界图片集来训练神经网络是，将不同大小的图片调整为相同大小显然是极其必要的

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_image(image_string)
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label

# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

或者依靠外部数据库来转化图片

In [ ]:
import cv2

# Use a custom OpenCV function to read the image, instead of the standard
# TensorFlow `tf.read_file()` operation.
def _read_py_function(filename, label):
    image_decoded = cv2.imread(filename.decode(), cv2.IMREAD_GRAYSCALE)
    return image_decoded, label

# Use standard TensorFlow operations to resize the image to a fixed shape.
def _resize_function(image_decoded, label):
    image_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label

filenames = ["/var/data/image1.jpg", "/var/data/image2.jpg", ...]
labels = [0, 37, 29, 1, ...]

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(
    lambda filename, label: tuple(tf.py_func(
        _read_py_function, [filename, label], [tf.uint8, label.dtype])))
dataset = dataset.map(_resize_function)

### 5.数据集的批处理
#### 5.1 简单的批处理
最简单的批处理就是将n个数据集中连续元素堆叠为一个元素。Dataset.batch（）就是这样做的转化工作，注意每个元素必须是完全相同的大小。

In [23]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

(array([0, 1, 2, 3]), array([ 0, -1, -2, -3]))
(array([4, 5, 6, 7]), array([-4, -5, -6, -7]))
(array([ 8,  9, 10, 11]), array([ -8,  -9, -10, -11]))


#### 5.2 不那么简单的批处理
恩上面的代码呢，要求每个元素必须完全相同大小，但实际情况中呢，许多模型的输入数据大小并不一定相同（例如不同长度的短句）。为了解决这个问题，Dataset.padded_batch()方法使你可以一次性批处理不同大小的tensors

In [24]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
dataset = dataset.padded_batch(4, padded_shapes=[None])

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> [[0, 0, 0], [1, 0, 0], [2, 2, 0], [3, 3, 3]]
print(sess.run(next_element))  # ==> [[4, 4, 4, 4, 0, 0, 0],
                               #      [5, 5, 5, 5, 5, 0, 0],
                               #      [6, 6, 6, 6, 6, 6, 0],
                               #      [7, 7, 7, 7, 7, 7, 7]]

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]


Note:Dataset.padded_batch()方法允许你为每个维度的每个组件设置不同的填充，可以使可变的长度（None），也可以是定值。
<br>
(It is also possible to override the padding value, which defaults to 0.这句话不知道怎么翻译没有写)

### 6.训练工作流
#### 6.1 Processing Multiple Epochs
tf.data提供两种主要方式来处理多次epochs训练
* 最简单的方式是使用Dataset.repeat()转化，例如，创建一个数据集重复10个epoch

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.repeat(10)
dataset = dataset.batch(32)

应用Dataset.repeat()，如果不设置的参数的话，将一直重复下去。且该方法在训练前后两个epoch时，不会在第一个epoch结束后发出信号.
<br>如果你想在每个epoch结束时收到信号，你可以写一个训练循环来捕捉tf.errors.OutOfRangeError在每个数据集的最后。在这一点你可能会收集到一些关于当前epoch的统计信息

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Compute for 100 epochs.
for _ in range(100):
    sess.run(iterator.initializer)
    while True:
    try:
          sess.run(next_element)
    except tf.errors.OutOfRangeError:
          break

  # [Perform end-of-epoch calculations here.]

#### 6.2 Randomly shuffling input data
(其实是打乱输入数据顺序的意思，但我不知道怎么翻译比较文雅)
<BR>
Dataset.shuffle()方法随机打乱输入数据用一个类似于tf.RandomShuffleQueue的算法：它维护一个固定大小的缓存，并且从缓存中随机选取下一个元素。

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat()

#### 6.3 调用高层接口
 tf.train.MonitoredTrainingSession简化了分布式运行Tensorflow的设置，MonitoredTrainingSession调用tf.errors.OutOfRangeError来提示训练已经结束，所以为了和tf.data接口能够兼容使用，我们(Tensorflow的开发员们)推荐使用Dataset.make_one_shot_iterator().例如：

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat(num_epochs)
iterator = dataset.make_one_shot_iterator()

next_example, next_label = iterator.get_next()
loss = model_function(next_example, next_label)

training_op = tf.train.AdagradOptimizer(...).minimize(loss)

with tf.train.MonitoredTrainingSession(...) as sess:
    while not sess.should_stop():
        sess.run(training_op)

为了在Estimator的input_fn函数中使用Dataset，我们也推荐使用Dataset.make_one_shot_iterator()，例如：

In [ ]:
def dataset_input_fn():
    filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
    dataset = tf.data.TFRecordDataset(filenames)

  # Use `tf.parse_single_example()` to extract data from a `tf.Example`
  # protocol buffer, and perform any additional per-record preprocessing.
    def parser(record):
        keys_to_features = {
        "image_data": tf.FixedLenFeature((), tf.string, default_value=""),
        "date_time": tf.FixedLenFeature((), tf.int64, default_value=""),
        "label": tf.FixedLenFeature((), tf.int64,
                                    default_value=tf.zeros([], dtype=tf.int64)),
        }
        parsed = tf.parse_single_example(record, keys_to_features)

    # Perform additional preprocessing on the parsed data.
        image = tf.image.decode_jpeg(parsed["image_data"])
        image = tf.reshape(image, [299, 299, 1])
        label = tf.cast(parsed["label"], tf.int32)

        return {"image_data": image, "date_time": parsed["date_time"]}, label

  # Use `Dataset.map()` to build a pair of a feature dictionary and a label
  # tensor for each example.
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(32)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()

  # `features` is a dictionary in which each value is a batch of values for
  # that feature; `labels` is a batch of labels.
    features, labels = iterator.get_next()
    return features, labels

好了，今日份的tutorial结束了，不得不说太长了，翻译起来真是麻烦T T